In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
data = requests.get("https://dl.acm.org/action/doSearch?AllField=%22human-ai+interaction%22&startPage=0&pageSize=50")

In [3]:
soap = BeautifulSoup(data.text, 'html.parser')

In [4]:
def parseDoi(doi):
    a = doi[5:]
    if(a[0] == 'b'):
        a = a[5:]
    return a

data = []
for entry in soap.find_all("div", attrs={"class": "issue-item__content"}):
    # print(entry)
    title = entry.find('h5', attrs={'class': 'issue-item__title'}).text
    doi = parseDoi(entry.find('a')['href'])
    # years = entry.find('div', attrs={'class': ['bookPubDate', 'simple-tooltip__block--b']})
    print(title, doi)
    data.append({
       "title": title,
       "doi": parseDoi(entry.a['href'])})

HAID: Human-AI Interaction for Dementia Care 10.1145/3677045.3685469
Theory of Mind in Human-AI Interaction 10.1145/3613905.3636308
Assessing Human-AI Interaction Early through Factorial Surveys: A Study on the Guidelines for Human-AI Interaction 10.1145/3511605
Spiritual AI: Exploring the Possibilities of a Human-AI Interaction Beyond Productive Goals 10.1145/3613905.3650743
Development and translation of human-AI interaction models into working prototypes for clinical decision-making 10.1145/3643834.3660697
Exploration of Explainable AI for Trust Development on Human-AI Interaction 10.1145/3639592.3639625
Re-examining User Burden in Human-AI Interaction: Focusing on a Domain-Specific Approach 10.1145/3613905.3638186
Past Meets Future: Human-AI Interaction for Digital History and Cultural Heritage 10.1145/3640544.3645257
A Systematic Review on Fostering Appropriate Trust in Human-AI Interaction: Trends, Opportunities and Challenges 10.1145/3696449
Human-AI interaction: intermittent, c

In [10]:
def get_data_from_semantic_scholar(doi):
    base_url = "https://api.semanticscholar.org/v1/paper/"
    url = f"{base_url}{doi}"
    headers = {"Accept": "application/json"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        abstract = data.get('abstract')
        year = data.get('year')
        
        if abstract:
            return [abstract, year]
        else:
            return ["", year]
    
    except requests.exceptions.RequestException as e:
        print(e)
        return ["error", ""]

In [11]:
import pandas as pd

df = pd.DataFrame()

for entry in data:
    fetched = get_data_from_semantic_scholar(entry["doi"])
    print("fetching", entry["title"])
    if fetched[0] != "error":
        df = df._append({
            "title": entry["title"],
            "year": fetched[1],
            "doi": entry["doi"],
            "abstract": fetched[0],
        }, ignore_index=True)

fetching HAID: Human-AI Interaction for Dementia Care
fetching Theory of Mind in Human-AI Interaction
fetching Assessing Human-AI Interaction Early through Factorial Surveys: A Study on the Guidelines for Human-AI Interaction
fetching Spiritual AI: Exploring the Possibilities of a Human-AI Interaction Beyond Productive Goals
fetching Development and translation of human-AI interaction models into working prototypes for clinical decision-making
fetching Exploration of Explainable AI for Trust Development on Human-AI Interaction
fetching Re-examining User Burden in Human-AI Interaction: Focusing on a Domain-Specific Approach
fetching Past Meets Future: Human-AI Interaction for Digital History and Cultural Heritage
fetching A Systematic Review on Fostering Appropriate Trust in Human-AI Interaction: Trends, Opportunities and Challenges
fetching Human-AI interaction: intermittent, continuous, and proactive
fetching Measuring User Experience Inclusivity in Human-AI Interaction via Five User 

In [13]:
# save df to csv
df.to_csv("acm.csv")